<a href="https://colab.research.google.com/github/cmannnn/practical_deep_learning_for_coders/blob/main/ch_4_digit_recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MNIST digit recognizer

In [1]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 720 kB 4.4 MB/s 
     |████████████████████████████████| 188 kB 44.8 MB/s 
     |████████████████████████████████| 1.2 MB 45.4 MB/s 
     |████████████████████████████████| 46 kB 1.3 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 
     |████████████████████████████████| 51 kB 324 kB/s 
Mounted at /content/gdrive


In [2]:
#hide
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

##Q1

Create your own implementation of Learner from scratch, based on the training loop shown in this chapter.

In [3]:
# setting the path to the MNIST dataset
path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path

In [22]:
# grabbing just the threes and sevens form the testing set to use
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()

In [24]:
# changing the images into tensors
three_tensor = [tensor(Image.open(o)) for o in threes]
seven_tensor = [tensor(Image.open(o)) for o in sevens]

In [18]:
# creating Pytorch tensors of the valid 3s and 7s
valid_3_tens = torch.stack([tensor(Image.open(o))
                            for o in (path/'valid'/'3').ls()])

valid_7_tens = torch.stack([tensor(Image.open(o))
                            for o in (path/'valid'/'7').ls()])

In [26]:
# stack then normalize the train 3 tensor
stacked_threes = torch.stack(three_tensor).float()/255

# stack then normalize the train 7 tensor
stacked_sevens = torch.stack(seven_tensor).float()/255

# normalizing the valid 3 tensor
valid_3_tens = valid_3_tens.float()/255

# normalizing the valid 7 tensor
valid_7_tens = valid_7_tens.float()/255

In [21]:
# changing training and valid data to rank 2 tensors
valid_3_tens.shape

torch.Size([1010, 28, 28])

In [ ]:
train_x = torch.cat([])

##Q2

Complete all the steps in this chapter using the full MNIST datasets (that is, for all digits, not just 3s and 7s). This is a significant project and will take you quite a bit of time to complete! 